# Try the RM demo with some modifications:
1. add pyvista for plotting

In [1]:
import numpy as np

import dolfinx
import ufl
from dolfinx.fem import Function, FunctionSpace, dirichletbc, Constant
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import CellType, create_unit_square
from ufl import (FiniteElement, MixedElement, VectorElement, dx, grad, inner,
                 split, sym, tr)

from mpi4py import MPI

## Create mesh

In [2]:
mesh = create_unit_square(MPI.COMM_WORLD, 32, 32, CellType.triangle)

## Define Elements, FunctionSpace, Functions 

In [3]:
U_el = MixedElement([VectorElement("Lagrange", ufl.triangle, 2), FiniteElement("Lagrange", ufl.triangle, 1),
                     FiniteElement("N1curl", ufl.triangle, 1), FiniteElement("N1curl", ufl.triangle, 1)])
U = FunctionSpace(mesh, U_el)

u_ = Function(U)
u = ufl.TrialFunction(U)
u_t = ufl.TestFunction(U)

theta_, w_, R_gamma_, p_ = split(u_)

## Material parameters

In [4]:
from dolfinx import default_scalar_type

E = Constant(mesh, default_scalar_type(10920.0))
nu = Constant(mesh, default_scalar_type(0.3))
kappa = Constant(mesh, default_scalar_type(5.0 / 6.0))
t = Constant(mesh, default_scalar_type(0.001))

## Bending strain and bending energy

Bending strain tensor k:
$$
 k(\theta) = \dfrac{1}{2}(\nabla \theta + (\nabla \theta)^T)
$$

Bending energy $\psi_b$ and bending modulus 
$$
 \psi_b(k) = \frac{1}{2} D \left( (1 - \nu) \, \mathrm{tr}\,(k^2) + \nu \, (\mathrm{tr}    \,k)^2 \right) \qquad
 D = \frac{Et^3}{12(1 - \nu^2)}
$$

In [5]:
k = sym(grad(theta_)) 

D = (E*t**3)/(12.0*(1.0 - nu**2))

psi_b = 0.5 * D*((1.0 - nu)*tr(k*k) + nu*(tr(k))**2)

## Shear energy

In [6]:
psi_s = ((E*kappa*t)/(4.0*(1.0 + nu)))*inner(R_gamma_, R_gamma_)

## External energy

In [7]:
W_ext = inner(1.0*t**3, w_)*dx

## Imposing MITC method weekly

In [8]:
gamma = grad(w_) - theta_

# internal Facets
dSp = ufl.Measure('dS', metadata={'quadrature_degree': 1}) 

# boundary Facets
dsp = ufl.Measure('ds', metadata={'quadrature_degree': 1})

# normal of the facet
n = ufl.FacetNormal(mesh)

# tangent of the facet
t = ufl.as_vector((-n[1], n[0]))

# imposing MITC conditions weekly with lagrange multipliers
# (+) is the DG operator
def inner_e(x, y):
    return (inner(x, t)*inner(y, t))('+') * \
        dSp + (inner(x, t)*inner(y, t))*dsp


Pi_R = inner_e(gamma - R_gamma_, p_)

## Jacobian and residual

In [9]:
Pi = psi_b*dx + psi_s*dx + Pi_R - W_ext
F = ufl.derivative(Pi, u_, u_t)
J = ufl.derivative(F, u_, u)

## Boundary conditions : all clamped

In [10]:
# x with shape (gdim, num_points)
def all_boundary(x):
    return np.full(x.shape[1], True, dtype=bool)

# In defaute, the function value is zero
u_boundary = Function(U)

boundary_entities = dolfinx.mesh.locate_entities_boundary(
    mesh, mesh.topology.dim - 1, all_boundary)

boundary_dofs = dolfinx.fem.locate_dofs_topological(
    U, mesh.topology.dim - 1, boundary_entities)

bcs = [dirichletbc(u_boundary, boundary_dofs)]

## Solving with Petsc 

In [11]:
problem = LinearProblem(J, -F, bcs=bcs, petsc_options={
                        "ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
u_ = problem.solve()

theta, w, R_gamma, p = u_.split()

## Testing the results

In [12]:
# bb_tree is a kind of data structure to quickly find the cells 
# that are close to a given point
bb_tree = dolfinx.geometry.bb_tree(mesh, 2)

# target point
point = np.array([[0.5, 0.5, 0.0]], dtype=np.float64)

# Find the leaf that the target point is in
cell_candidates = dolfinx.geometry.compute_collisions_points(bb_tree, point)

# Find the cell from the leaf that contains the target point
cells = dolfinx.geometry.compute_colliding_cells(
    mesh, cell_candidates, point)

if len(cells) > 0:
    value = w.eval(point, cells.array[0])
    print(value[0])
    # NOTE: FEniCS-Shells (old dolfin) `demo/documented/reissner-mindlin-clamped`
    # gives 1.28506469462e-06 on a 32 x 32 mesh and 1.2703580973e-06 on a 64 x 64
    # mesh.

    def test_center_displacement():
        assert np.isclose(value[0], 1.285E-6, atol=1E-3, rtol=1E-3)
    test_center_displacement()

1.2850646949512924e-06


## Write results to files

In [13]:
from dolfinx import io
from pathlib import Path
results_folder = Path("results/RM_clamped")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "RM_clamped"

with io.VTXWriter(mesh.comm, filename.with_suffix(".bp"), [w]) as vtx:
    vtx.write(0.0)

with io.XDMFFile(mesh.comm, filename.with_suffix(".xdmf"), "w") as f:
    f.write_mesh(mesh)
    f.write_function(w)

## Plotting with pyvista

In [16]:
import pyvista
from dolfinx import plot

W = FunctionSpace(mesh, FiniteElement("Lagrange", ufl.triangle, 1))
pyvista.start_xvfb()
topology, cell_types, geometry = plot.vtk_mesh(W)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
w_collapsed = w.collapse()
grid.point_data["w"] = w_collapsed.x.array
grid.set_active_scalars("w")

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()


Widget(value='<iframe src="http://localhost:34967/index.html?ui=P_0x7f7a05f8be30_2&reconnect=auto" class="pyvi…

In [17]:
warped = grid.warp_by_scalar("w", factor=250000)
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)

plotter2.show()


Widget(value='<iframe src="http://localhost:34967/index.html?ui=P_0x7f7ab7de3590_3&reconnect=auto" class="pyvi…